# Example of DOV search methods for hydrogeological interpretations

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/DOV-Vlaanderen/pydov/master?filepath=docs%2Fnotebooks%2Fsearch_hydrogeologische_stratigrafie.ipynb)

## Use cases:
* Select interpretations in a bbox
* Select interpretations in a bbox with selected properties
* Select interpretations in a municipality
* Get interpretations using info from wfs fields, not available in the standard output dataframe

In [1]:
%matplotlib inline
import os, sys
import inspect

In [3]:
import pydov

## Get information about code base

In [4]:
from pydov.search.interpretaties import HydrogeologischeStratigrafieSearch
ip_hydrogeo = HydrogeologischeStratigrafieSearch()

# information about the HydrogeologischeStratigrafie type (In Dutch):
print(ip_hydrogeo.get_description())

De hydrostratigrafie geeft, op basis van de (gecodeerde) lithologie, een indeling weer naar de al dan niet watervoerende eigenschappen van een bepaald beschreven diepte-interval. Deze interpretatie respecteert de lithostratigrafie van het Tertiair, maar deelt deze anders in. De hiervoor gebruikte standaard is de Hydrogeologische Codering van de Ondergrond van Vlaanderen (HCOV). Deze kan beschouwd worden als de officiele hydrogeologische codering voor het Vlaams Gewest.


In [5]:
# information about the available fields for a HydrogeologischeStratigrafie object
fields = ip_hydrogeo.get_fields()
# print available fields
for f in fields.values():
    print(f['name'])

pkey_interpretatie
Type_proef
Proefnummer
pkey_boring
x
y
Z_mTAW
diepte_tot_m
gemeente
Auteurs
Datum
Opdrachten
betrouwbaarheid_interpretatie
Geldig_van
Geldig_tot
diepte_laag_van
diepte_laag_tot
aquifer


In [6]:
# print information for a certain field
fields['aquifer']

{'name': 'aquifer',
 'type': 'string',
 'definition': 'code van de watervoerende laag waarin de laag Hydrogeologische stratigrafie zich bevindt.',
 'notnull': False,
 'query': False,
 'cost': 10,
 'values': {'0000': 'Onbekend',
  '0100': 'Quartaire aquifersystemen',
  '0110': 'Ophogingen',
  '0120': 'Duinen',
  '0130': 'Polderafzettingen',
  '0131': 'Kleiige polderafzettingen van de kustvlakte',
  '0132': 'Kleiige polderafzettingen van het Meetjesland',
  '0133': 'Kleiige polderafzettingen van Waasland-Antwerpen',
  '0134': 'Zandige kreekruggen',
  '0135': 'Veen-kleiige poelgronden',
  '0140': 'Alluviale deklagen',
  '0150': 'Deklagen',
  '0151': 'Zandige deklagen',
  '0152': 'Zand-lemige deklagen',
  '0153': 'Lemige deklagen',
  '0154': 'Kleiige deklagen',
  '0160': 'Pleistocene afzettingen',
  '0161': 'Pleistoceen van de kustvlakte',
  '0162': 'Pleistoceen van de Vlaamse Vallei',
  '0163': 'Pleistoceen van de riviervalleien',
  '0170': 'Maas- en Rijnafzettingen',
  '0171': 'Afzetting

The cost is an arbitrary attribute to indicate if the information is retrieved from a wfs query (cost = 1), 
or from an xml (cost = 10)

In [7]:
# if an attribute can have several values, these are listed under 'values', e.g. for 'Type_proef':
fields['Type_proef']

{'name': 'Type_proef',
 'definition': "Het type van de proef waarvan de hydrogeologische stratigrafie een interpretatie is. Bij een hydrogeologische stratigrafie zal dit altijd 'Boring' zijn.",
 'type': 'string',
 'notnull': False,
 'query': True,
 'cost': 1,
 'values': {'Boring': 'De hydrogeologische stratigrafie is een interpretatie van een boring.'}}

## Try-out of use cases

### Select interpretations in a bbox

In [8]:
from pydov.util.location import Within, Box

# Get all borehole data in a bounding box (llx, lly, ulx, uly)
# the pkey_boring link is not available below, but is in the df
df = ip_hydrogeo.search(location=Within(Box(153145, 206930, 153150, 206935)))
print(df.head())

[000/001] .
                                  pkey_interpretatie  \
0  https://www.dov.vlaanderen.be/data/interpretat...   
1  https://www.dov.vlaanderen.be/data/interpretat...   

                                         pkey_boring  \
0  https://www.dov.vlaanderen.be/data/boring/1974...   
1  https://www.dov.vlaanderen.be/data/boring/1974...   

  betrouwbaarheid_interpretatie         x         y  diepte_laag_van  \
0                          goed  153147.0  206931.0              0.0   
1                          goed  153147.0  206931.0              2.5   

   diepte_laag_tot aquifer  
0              2.5    0152  
1              7.0    0254  


### Select interpretations in a bbox with selected properties

In [9]:
# list available query methods
methods = [i for i,j in inspect.getmembers(sys.modules['owslib.fes'], 
                                           inspect.isclass) 
           if 'Property' in i]
print(methods)
from owslib.fes import PropertyIsGreaterThanOrEqualTo

['PropertyIsBetween', 'PropertyIsEqualTo', 'PropertyIsGreaterThan', 'PropertyIsGreaterThanOrEqualTo', 'PropertyIsLessThan', 'PropertyIsLessThanOrEqualTo', 'PropertyIsLike', 'PropertyIsNotEqualTo', 'PropertyIsNull', 'SortProperty']


The property feature methodes listed above are available from the owslib module. These were not adapted for use in pydov.

In [10]:
# Get deep boreholes in a bounding box 
from owslib.fes import PropertyIsEqualTo
# the propertyname can be any of the fields of the hydrogeological interpretations object that belong to the wfs source
# the literal is always a string, no matter what its definition is in the boring object (string, float...)
query = PropertyIsGreaterThanOrEqualTo(
        propertyname='betrouwbaarheid_interpretatie', literal='goed')
df = ip_hydrogeo.search(location=Within(Box(153145, 206930, 153150, 206935)),
                        query=query
                       )
print(df.head())

[000/001] c
                                  pkey_interpretatie  \
0  https://www.dov.vlaanderen.be/data/interpretat...   
1  https://www.dov.vlaanderen.be/data/interpretat...   

                                         pkey_boring  \
0  https://www.dov.vlaanderen.be/data/boring/1974...   
1  https://www.dov.vlaanderen.be/data/boring/1974...   

  betrouwbaarheid_interpretatie         x         y  diepte_laag_van  \
0                          goed  153147.0  206931.0              0.0   
1                          goed  153147.0  206931.0              2.5   

   diepte_laag_tot aquifer  
0              2.5    0152  
1              7.0    0254  


### Select interpretations in a municipality

In [11]:
query = PropertyIsEqualTo(propertyname='gemeente',
                          literal='Aartselaar')
df = ip_hydrogeo.search(query=query)
print(df.head())

[000/006] ......
                                  pkey_interpretatie  \
0  https://www.dov.vlaanderen.be/data/interpretat...   
1  https://www.dov.vlaanderen.be/data/interpretat...   
2  https://www.dov.vlaanderen.be/data/interpretat...   
3  https://www.dov.vlaanderen.be/data/interpretat...   
4  https://www.dov.vlaanderen.be/data/interpretat...   

                                         pkey_boring  \
0  https://www.dov.vlaanderen.be/data/boring/1950...   
1  https://www.dov.vlaanderen.be/data/boring/1950...   
2  https://www.dov.vlaanderen.be/data/boring/1895...   
3  https://www.dov.vlaanderen.be/data/boring/1895...   
4  https://www.dov.vlaanderen.be/data/boring/1894...   

  betrouwbaarheid_interpretatie         x         y  diepte_laag_van  \
0                          goed  151457.0  201527.0              0.0   
1                          goed  151457.0  201527.0             49.0   
2                          goed  151041.0  202065.0              0.0   
3                    

### Get interpretations using info from wfs fields, not available in the standard output dataframe

To keep the dataframe size acceptable, not all wfs fields are included in the standard output. However, one can use this information to select interpretations as illustrated below.
The available wfs fields can be viewed in the object catalog of the 'Hydrogeologische stratigrafie' wfs service, on-line [2018 05 16]: https://www.dov.vlaanderen.be/geonetwork/apps/tabsearch/index.html?hl=dut.
For example, make a selection of the interpretations in the municipality of Antwerp that were derived from cpt data

In [12]:
from owslib.fes import And
from owslib.fes import PropertyIsEqualTo
query = And([PropertyIsEqualTo(propertyname='gemeente',
                               literal='Antwerpen'),
             PropertyIsEqualTo(propertyname='Type_proef', 
                               literal='Boring')]
            )
df = ip_hydrogeo.search(query=query, )
print(df.head())

[000/191] ..................................................
[050/191] ..................................................
[100/191] ...............................................c..
[150/191] .........................................
                                  pkey_interpretatie  \
0  https://www.dov.vlaanderen.be/data/interpretat...   
1  https://www.dov.vlaanderen.be/data/interpretat...   
2  https://www.dov.vlaanderen.be/data/interpretat...   
3  https://www.dov.vlaanderen.be/data/interpretat...   
4  https://www.dov.vlaanderen.be/data/interpretat...   

                                         pkey_boring  \
0  https://www.dov.vlaanderen.be/data/boring/1956...   
1  https://www.dov.vlaanderen.be/data/boring/1956...   
2  https://www.dov.vlaanderen.be/data/boring/1956...   
3  https://www.dov.vlaanderen.be/data/boring/1979...   
4  https://www.dov.vlaanderen.be/data/boring/1979...   

  betrouwbaarheid_interpretatie         x         y  diepte_laag_van  \
0                  

### Visualize results

Using Folium, we can display the results of our search on a map.

In [16]:
# import the necessary modules (not included in the requirements of pydov!)
import folium
from folium.plugins import MarkerCluster
from pyproj import Transformer

In [17]:
# convert the coordinates to lat/lon for folium
def convert_latlon(x1, y1):
    transformer = Transformer.from_crs("epsg:31370", "epsg:4326", always_xy=True)
    x2,y2 = transformer.transform(x1, y1)
    return x2, y2

df['lon'], df['lat'] = zip(*map(convert_latlon, df['x'], df['y'])) 
# convert to list
loclist = df[['lat', 'lon']].values.tolist()

In [18]:
# initialize the Folium map on the centre of the selected locations, play with the zoom until ok# initia 
fmap = folium.Map(location=[df['lat'].mean(), df['lon'].mean()], zoom_start=12)
marker_cluster = MarkerCluster().add_to(fmap)
for loc in range(0, len(loclist)):
    folium.Marker(loclist[loc], popup=df['aquifer'][loc]).add_to(marker_cluster)
fmap